# Context
Notebook for analyzing the data collected on 2023-12-21. Hardware:
* Philips patient monitor (used during Kapiolani study)
* Accelerometer (placed near the intercostal muscles, and collected via putty)

Goal:
1. Does the chest accelerometer provide enough breathing info?
2. Can manual counting match the breathing rate collected on children (age 5)?

In [ ]:
from ppg2rr import import_ppg
from ppg2rr.util import lowpass_butter
from ppg2rr import util
from scipy import signal
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import glob
import pprint


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = '../data/Philips/2023-12-20 Folk Chest Accel Position2'

# Analyze Data

In [ ]:
experiment_id = 'S-41' # 'S-41' and 'S-42'

ppg_raw, fs_ppg, ref = import_ppg.load_philips_monitor_data(
    root_dir=data_dir,
    folder_id=experiment_id,
    probe_num = 1,
    show=True,
)

In [ ]:
# ppg collected from the new horizons device
nh_datafile = glob.glob(f"{data_dir}/{experiment_id}/*.csv")
nh_df = pd.read_csv(nh_datafile[0])
nh_df[['ppg1','ppg2','ppg_in_waiting','elapsed_time_ms']]

fig = go.Figure()
ppg_t = np.arange(len(nh_df))/250
fig.add_scatter(
    # x=nh_df['elapsed_time_ms'],
    x=ppg_t,
    y=nh_df['ppg1']-np.mean(nh_df['ppg1']),
    mode='lines',
    name='PPG_R'
    )
fig.add_scatter(
    x=ppg_t,
    y=nh_df['ppg2']-np.mean(nh_df['ppg2']),
    mode='lines',
    name='PPG_IR'
    )

In [ ]:
from ppg2rr.capnography import rr_from_capnography

x_capnography, rr_capnography = rr_from_capnography(ref['co2']['y'], fs_co2=62, show=True)
print(f'Mean estiamted RR by capnography algorithm: {np.mean(rr_capnography):.1f}')
print(f'Total Breath Count by capnography algorithm: {(len(x_capnography)+1)}')
print(f'Breathing rate by counting capnography waveform: {(len(x_capnography)+1)*60/ref["co2"]["x"][-1]:.1f} bpm')


# Algorithm Estimate

In [ ]:
# algorithm estimates from philips finger probe
from ppg2rr import rr_est
(
    rr_candidate_merged,
    all_rr_candidates,
    feature_quality,
    hr
) = rr_est.estimate_rr_single_frame(
    ppg=ppg_raw,
    fs_ppg = 125,
    min_resp_freq = 0.1,
    show=False
)

In [ ]:
rr_candidate_merged

In [ ]:
# algorithm estimates from NH ppg device (channel 1)
(
    rr_candidate_merged,
    all_rr_candidates,
    feature_quality,
    hr
) = rr_est.estimate_rr_single_frame(
    ppg=nh_df['ppg1'],
    fs_ppg = 250,
    min_resp_freq = 0.1,
    show=False
)
pprint.pprint(rr_candidate_merged)
feature_quality[-9]

In [ ]:
# algorithm estimates from NH ppg device (channel 2)
(
    rr_candidate_merged,
    all_rr_candidates,
    feature_quality,
    hr
) = rr_est.estimate_rr_single_frame(
    ppg=nh_df['ppg2'],
    fs_ppg = 250,
    min_resp_freq = 0.1,
    show=False
)
pprint.pprint(rr_candidate_merged)
feature_quality[-9]

The mean and median of fused candidates are calcualted using the following:
```
[
        "PSD, closest to prev RR",
        "PSD median",
        "Counting, median pk delta std cutoff",
        "kalman",
]
```
And has been returning the best results. This method correctly estimates the right breathing rate for S-31.

# Accelerometer Data

In [ ]:
# load accel data, do some processing and compute magnutide
# The 2000 multiplier is used to match the capnography amplitude
df_acc = pd.read_csv(f'{data_dir}/{experiment_id}/{experiment_id} - Acc', skiprows=1)
df_acc=df_acc.drop(columns='0')
df_acc.columns=['x', 'y', 'z']
for col in df_acc.columns:
    df_acc[f'{col}_medfilt'] = util.normalize(signal.medfilt(df_acc[col],kernel_size=5))*2000
    
df_acc['magnitude'] = np.linalg.norm(df_acc[['x_medfilt', 'y_medfilt', 'z_medfilt']], axis=1)

In [ ]:
len(df_acc['magnitude'])/60

In [ ]:
px.line(df_acc)

In [ ]:
# lowpass the acceleration

# Take the first 30 seconds of the signal
# Assume 60 hz sampling rate
magnitude_lp = lowpass_butter(
    signal_raw=df_acc['magnitude'][:125*30].dropna(),
    signal_fs = 60,
    f_max=2
)

fig = px.line(magnitude_lp, labels='2z LPF accel magnitude')

fig.data[0].name = 'accel magnitude<br>LPF 2hz'
fig.update_layout(title="Processed accel magnitude provides enough information to identify breaths")
fig.show()

In [ ]:
# rescale each signal for plotting
for col in df_acc.columns:
    df_acc[col] = util.normalize(df_acc[col],range=(-200,2000))
    
fig = go.Figure()
fig.add_scatter(
    x=ref['co2']['x'],
    y=ref['co2']['y'],
    mode='lines',
    name='capnography'
    )
accel_t = np.arange(len(df_acc))/60
fig.add_scatter(
    x = accel_t+9,
    y = df_acc['x_medfilt']-2000,
    name = 'accel x'
)
fig.add_scatter(
    x = accel_t+9,
    y = df_acc['y_medfilt']-3000,
    name = 'accel y'
)
fig.add_scatter(
    x = accel_t+9,
    y = df_acc['z_medfilt']-4000,
    name = 'accel z'
)
fig.add_scatter(
    x = accel_t+9,
    y = df_acc['magnitude']-1000,
    name = 'accel norm'
)
fig.add_scatter(
    x = accel_t+9,
    y = magnitude_lp-500,
    name = 'accel norm lowpass'
)
fig.update_xaxes(title="estimated time (seconds)")

In [ ]:
from scipy import signal

In [ ]:
f, pxx = signal.periodogram(magnitude_lp, fs=60)
fig = px.line(y=pxx, x=f*60)
fig.update_layout(
    xaxis_title="frequency (breaths per minute)",
    yaxis_title="accelerometer, power spectra",
    )

Accel's Frequency spectra is inconclusive

In [ ]:
f, pxx = signal.periodogram(ref['co2']['y'][:round(30*62.5)], fs=62.5)
fig = px.line(y=pxx, x=f*60)
fig.update_layout(
    xaxis_title="frequency (breaths per minute)",
    yaxis_title="Capnography, power spectra",
    )


Capnography shows frequency peaks at 32 and 14 breaths per minute for the first 30 seconds.

# Manual Counting

In [ ]:
# Craig's manual annotation
estimated_timings_1 = [1, 2, 3, 4, 5]


# Plot manual count with Capnography trace
fig = go.Figure()
fig.add_scatter(
    x=ref['co2']['x'],
    y=ref['co2']['y'],
    mode='lines',
    name='capnography'
    )
for timestamp in estimated_timings_1:
    fig.add_vline(
        x=timestamp,
        line_dash="dash"
        )
fig.update_layout(
    xaxis_title="time (seconds)",
    title="Capnography vs Video time stamps"
)
fig.show()

# total breath counts
len(estimated_timings_1)